- #### Transformaciones
    - ##### sort/orderBy
    - ##### dropDuplicates
    - ##### distinct
    - ##### groupBy
    - ##### union/unionAll
    - ##### unionByName

In [ ]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder
        .appName("sesion_1")
        .master("local[*]")
        .getOrCreate()
val sc = spark.sparkContext

In [ ]:
sc.uiWebUrl

In [ ]:
val BasePath = "../../resources/data/parquet/"
val contractsDf = spark.read.parquet(BasePath + "contracts_tmp")

contractsDf.show()
contractsDf.printSchema()

In [ ]:
// sort/orderBy

import org.apache.spark.sql.{functions => f}

contractsDf.orderBy(f.col("cod_iuc").desc, f.col("fec_alta_fin").desc).show()

In [ ]:
// dropDuplicates distinct

contractsDf.dropDuplicates(Seq("cod_iuc", "cod_producto")).show()

contractsDf.distinct().show()

In [ ]:
// groupBy -> count, max, min, avg, mean, sum

//contractsDf.groupBy("cod_iuc").count().show()
//contractsDf.groupBy(f.col("cod_iuc")).sum("diff").show()
//contractsDf.groupBy(f.col("cod_iuc")).max("fec_alta_ini").show()

contractsDf.groupBy(f.col("cod_iuc"), f.col("activo"))
    .agg(
        f.count("*").alias("count"),
        f.sum("diff").alias("sum_diff"),
        f.max("diff").alias("max_diff"),
        f.min("diff").alias("min_diff"),
        f.mean("diff").alias("mean_diff"),
        f.avg("diff").alias("avg_diff"),
        f.max("fec_alta_ini").alias("max_fec_alta_ini")
    ).show()

In [ ]:
contractsDf
    .groupBy("cod_iuc")
    .pivot("activo")
    .agg(
        f.count("*"),
        f.max("fec_alta_fin"),
        f.avg("diff")
    )
    .show()

In [ ]:
// union/unionAll unionByName
val contractsTmpDf = contractsDf.filter(f.col("activo").isNull)

contractsTmpDf.show()

contractsTmpDf.select("cod_titular","fec_alta_ini","actual_date")
    .union(contractsTmpDf.select("cod_titular","fec_alta_ini","actual_date")).show()

contractsTmpDf.select("cod_titular","fec_alta_ini","actual_date")
    .unionByName(contractsTmpDf.select("actual_date","cod_titular","fec_alta_ini")).show()